# User Interface 

## Initilize and import

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import ipywidgets as widgets
import rospy
import time
import actionlib
from std_msgs.msg import String
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from ipywidgets import Button,GridBox,Layout,FloatSlider,ButtonStyle
import rt2_assignment1.msg

#init node
rospy.init_node('user_interface')

# velocities publisher
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
vel_pub = rospy.Publisher('velocities', Twist, queue_size=10)
msg = Twist()

# user interface client
ui_client = rospy.ServiceProxy('/user_interface', Command)

#action client
act_client = actionlib.SimpleActionClient('position',rt2_assignment1.msg.PositionAction)
if(act_client.wait_for_server()):
    print("Connected to Action Server")
else:
    print("Unable to connect to Action Server")

Connected to Action Server


## Buttons to start and stop random target behavior

In [2]:
start_but = Button(description = "START", layout = Layout(grid_area='start_but'), style = ButtonStyle(button_color ='green'))
stop_but = Button(description = "STOP", layout = Layout(grid_area='stop_but'), style = ButtonStyle(button_color ='red'))

def on_start_clicked(b):
    global ui_client
    ui_client('start'
             )
    print("start")

def on_stop_clicked(b):
    global ui_client
    act_client.cancel_all_goals()
    ui_client('stop')
    print("goal canceled")

start_but.on_click(on_start_clicked)
stop_but.on_click(on_stop_clicked)

# show buttons in a grid Box
gridBox = GridBox(children=[start_but, stop_but],
                         layout = Layout(
                            width = '50%',
                            grid_template_rows = '10%',
                            grid_template_columns = '50% 50%',
                            grid_template_areas = ''' "start_but stop_but" '''
                         )
                    )
display(gridBox)


GridBox(children=(Button(description='START', layout=Layout(grid_area='start_but'), style=ButtonStyle(button_c…

start


## Slider to set linear and angular velocity

In [3]:
# slider for lin vel
lin = FloatSlider(min=0.0, max=2.0,value = 1.0)
display(lin)

#slider for ang vel
ang = FloatSlider(min=0.0, max=6.0, value = 3.0)
display(ang)

# fun to change lin vel ( maybe could send a twist on a topic and go to point could read it to set vel)
def on_value_change(change):
    global msg
    msg.linear.x = change['new']
    vel_pub.publish(msg)

# fun to change ang vel
def on_valueang_change(change):
    global msg
    msg.angular.z = change['new']
    vel_pub.publish(msg)

lin.observe(on_value_change, names='value')
ang.observe(on_valueang_change, names='value')

FloatSlider(value=1.0, max=2.0)

FloatSlider(value=3.0, max=6.0)

## Buttons to control the robot without random behavior

In [4]:
# declaration buttons
stop = Button(description = "STOP", layout= Layout(width='auto', grid_area='stop'))
forward = Button(description = "^", layout= Layout(width='auto', grid_area='forward'))
backward = Button(description = "v", layout= Layout(width='auto', grid_area='backward'))
right = Button(description = ">", layout= Layout(width='auto', grid_area='right'))
left = Button(description = "<", layout= Layout(width='auto', grid_area='left'))

# defining on_clicked functions
def on_stop_clicked(b):
    global msg,ui_client
    act_client.cancel_all_goals()
    ui_client('stop')
    msg.linear.x = 0
    msg.angular.z = 0
    pub.publish(msg)
    
def on_forward_clicked(b):
    global lin,msg,pub,ui_client
    act_client.cancel_all_goals()
    ui_client('stop')
    msg.linear.x = lin.value*0.3
    msg.angular.z = 0
    pub.publish(msg)
    
def on_backward_clicked(b):
    global lin,msg,pub,ui_client
    act_client.cancel_all_goals()
    ui_client('stop')
    msg.linear.x = -lin.value*0.3
    msg.angular.z = 0
    pub.publish(msg)
    
def on_left_clicked(b):
    global msg,ang,pub,ui_client
    act_client.cancel_all_goals()
    ui_client('stop')
    msg.linear.x = 0
    msg.angular.z = -ang.value*3.0
    pub.publish(msg)
    
def on_right_clicked(b):
    global msg,ang,pub,ui_client
    act_client.cancel_all_goals()
    ui_client('stop')
    msg.linear.x = 0
    msg.angular.z = ang.value*3.0
    pub.publish(msg)
    
# attach functions to buttons
stop.on_click(on_stop_clicked)
forward.on_click(on_forward_clicked)
backward.on_click(on_backward_clicked)
left.on_click(on_left_clicked)
right.on_click(on_right_clicked)

# show buttons in a grid Box
gridBox_buttons = GridBox(children=[stop, forward, backward, right, left],
                         layout = Layout(
                            width = '400px',
                            height = '100px',
                            grid_template_rows = '40% 40% 40%',
                            grid_template_columns = '20% 20% 20%',
                            grid_template_areas ='''
                            ". forward ."
                            "left stop right"
                            " . backward ."
                            '''
                         )
                    )
display(gridBox_buttons)

GridBox(children=(Button(description='STOP', layout=Layout(grid_area='stop', width='auto'), style=ButtonStyle(…

## Line plot cmd_vel VS actual velocity

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import ipywidgets as widgets
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist

#set up the figure
fig_vel = plt.figure(1)

#set up axis
ax_lin = fig_vel.add_subplot(211)
ax_ang = fig_vel.add_subplot(212)

#lines
line_odom_lin, = ax_lin.plot([], [], label= "odom lin vel")
line_odom_ang, = ax_ang.plot([], [], label= "odom ang vel")
line_cmd_lin, = ax_lin.plot([],[], label= "cmd vel lin")
line_cmd_ang, = ax_ang.plot([],[], label= "cmd ang vel")

#axes features
ax_lin.grid()
ax_ang.grid()

ax_lin.legend()
ax_ang.legend()

ax_lin.set_title("linear velocities")
ax_ang.set_title("angular velocities")

ax_lin.set(xlabel = "Time", ylabel = "Magnitude")
ax_ang.set(xlabel = "Time", ylabel = "Magnitude")

plots = [line_odom_lin,line_odom_ang,line_cmd_lin,line_cmd_ang]

# data
x_odom_data = []
x_cmd_data = []
vlin_odom_data = []
vang_odom_data = []
vlin_cmd_data = []
vang_cmd_data = []


# callbacks
def odom_callback(msg_odom):
    global vlin_odom_data, vang_odom_data, x_odom_data
    
    vlin_odom_data.append(msg_odom.twist.twist.linear.x)
    vang_odom_data.append(msg_odom.twist.twist.angular.z)
    
    # set x data on ax_lin 
    if(len(x_odom_data) != 0):
        x_odom_data.append(x_odom_data[len(x_odom_data)-1]+0.05)
    else:
        x_odom_data.append(0.05)
    

def cmd_vel_callback(msg_cmd):
    global vlin_cmd_data, vang_cmd_data, x_cmd_data
    
    vlin_cmd_data.append(msg_cmd.linear.x)
    vang_cmd_data.append(msg_cmd.angular.z)

    # set x data on ax_ang
    if(len(x_cmd_data) != 0):
        x_cmd_data.append(x_cmd_data[len(x_cmd_data)-1]+0.05)
    else:
        x_cmd_data.append(0.05)
        
# initialize the plot
def init_vel():
    ax_lin.set_xlim (0,10) # time axis
    ax_ang.set_xlim (0,10) # time axis
    
    ax_lin.set_ylim(-0.8,0.8) #vel axis
    ax_ang.set_ylim(-0.8,0.8) #vel axis
    
    return plots,

def animate_vel(frame):   
    
    #update x axis ax_odom
    ax_odom_min, ax_odom_max = ax_lin.get_xlim()
    
    if len(x_odom_data) != 0 and ax_odom_max < x_odom_data[len(x_odom_data)-1]:
        ax_lin.set_xlim(ax_odom_max, ax_odom_max+10)
        ax_lin.figure.canvas.draw()
        
    #update x axis ax_cmd
    ax_cmd_min,ax_cmd_max = ax_ang.get_xlim()
    
    
    if len(x_cmd_data) != 0 and ax_cmd_max < x_cmd_data[len(x_cmd_data)-1]:
        ax_ang.set_xlim(ax_cmd_max, ax_cmd_max+10)
        ax_ang.figure.canvas.draw()
        
    #update odom data
    line_odom_lin.set_data(x_odom_data,vlin_odom_data)
    line_odom_ang.set_data(x_odom_data,vang_odom_data)
    
    #update cmd data
    line_cmd_lin.set_data(x_cmd_data,vlin_cmd_data)
    line_cmd_ang.set_data(x_cmd_data,vang_cmd_data)    
    
    return plots,
    

#subscribers
sub_odom = rospy.Subscriber('/odom', Odometry, odom_callback)
sub_cmd_vel = rospy.Subscriber('/cmd_vel', Twist, cmd_vel_callback)  

# show the plots
anim = animation.FuncAnimation(fig_vel, animate_vel, init_func = init_vel)
fig_vel.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Number of reached and canceled target

In [4]:
# plot for targets
fig_targets = plt.figure(2)
ax_targets = fig_targets.add_subplot()
ax_targets.set_title("number of targets reached or canceled")

#data for targets
num_reached = 0
num_canceled = 0
bars = [0,0]
labels = ["Targets reached","Target canceled"]
col = ["green", "red"]
bar_plot = ax_targets.bar(labels,bars,color = col)

def action_status_CB(msg):
    global num_reached, num_canceled, bar_plot
    
    print("target read: " + msg.data)
    
    if(msg.data == "SUCCEEDED"):
        num_reached += 1
        print("num_reached: " + str(num_reached))
        
        #show target plot updated
        bar_plot.remove()
        rospy.sleep(0.3)
        bars[0] = num_reached
        bar_plot = ax_targets.bar(labels,bars,color = col)        
        
    elif(msg.data == "PREEMPTED"):
        num_canceled += 1  
        print("num_canceled: " + str(num_canceled))
        
        #show target plot updated
        bar_plot.remove()
        rospy.sleep(0.3)
        bars[1] = num_canceled
        bar_plot = ax_targets.bar(labels,bars,color = col)

# status subscriber
sub_target = rospy.Subscriber('action_status', String, action_status_CB)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## time required to reach the targets

In [5]:
#plot for time to reach targets
fig_time = plt.figure(3)
ax_time = fig_time.add_subplot()
ax_time.set_title("histogram of times to reach targe")

# data for time to reach targets
start_time = rospy.Time()
arrival_time = rospy.Time()
time_to_reach = []
HIST_BINS = 120 # 120 to have a resolution of half of a second in the plot
_,_,hist_bars = ax_time.hist(time_to_reach,HIST_BINS, color = "skyblue")


def action_status_CB_time(msg):
    global start_time, arrival_time, time_to_reach, hist_bars
    
    print("time read: " + msg.data)
    
    if(msg.data == "SUCCEEDED"):
        #take arrival time
        arrival_time = rospy.get_rostime()
        time_to_reach.append(arrival_time.to_sec() - start_time.to_sec())
        print("time to reach a goal: "+ str(time_to_reach))

        #update the histogram of times to reach
        [b.remove() for b in hist_bars]
        _,_,hist_bars = ax_time.hist(time_to_reach,HIST_BINS) 
        
    elif(msg.data == "PENDING"):
        #take starting time
        start_time = rospy.get_rostime()

# status subscriber
sub_time = rospy.Subscriber('action_status', String, action_status_CB_time)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Show robot position and orientation

In [7]:
%matplotlib widget
# First set up the figure and the axis 
fig_odom = plt.figure(4)
ax_odom = fig_odom.add_subplot()
ax_odom.grid()

line, = plt.plot([], [], 'ro')
x_data = [] 
y_data = []

def init_odom():
    ax_odom.set_xlim(-5,5)
    ax_odom.set_ylim(-5,5)
    return line,

def odom_callback(msg):
    x_data.append(msg.pose.pose.position.x)
    y_data.append(msg.pose.pose.position.y)

def animate_odom(frame):
    line.set_data(x_data,y_data)
    return line,

sub_pos = rospy.Subscriber('odom', Odometry, odom_callback)

ani = animation.FuncAnimation(fig_odom, animate_odom, init_func = init_odom)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …